## Examples

### `Block`

In [1]:
from pathlib import Path
from src.blocks import BlockAnalyzer, Block, block_decomposition, Decomposer

In [4]:
Block(K=[1,2,3],i=2,j=7,label="ACGTAC").to_positional_string()

PositionalString(b='ACGTAC', i=2, j=7)

In [2]:
path_blocks = Path("out/blocks/Cluster_10644.json")
list_blocks = BlockAnalyzer()._load_list_blocks(path_blocks)

In [3]:
list_blocks

[Block(K=(1, 2), i=0, j=75, label='CACGATTCTCCCTTTGAGTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGC'),
 Block(K=(0, 1, 2), i=0, j=15, label='CACGATTCTCCCTTTG'),
 Block(K=(0, 1), i=17, j=81, label='GTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGCGAAAAA'),
 Block(K=(0, 1, 2), i=17, j=75, label='GTTGATGAGGTTTCAGGGAAAAGGATAGCTGATTCTCCGCTTTTGCAAGTATGAAAGGC'),
 Block(K=(0, 1, 2), i=77, j=81, label='AAAAA')]

In [4]:
decomposer=Decomposer()
new_blocks = decomposer(list_blocks)

In [5]:
len(new_blocks), len(list_blocks)

(9, 5)